# LookBench Quickstart Guide

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SerendipityOneInc/look-bench/blob/main/notebooks/01_quickstart.ipynb)

This notebook demonstrates how to get started with **LookBench** for fashion image retrieval evaluation.

📄 **Paper**: [LookBench: A Live and Holistic Open Benchmark for Fashion Image Retrieval](https://arxiv.org/abs/2601.14706)

🏠 **Project Page**: [https://serendipityoneinc.github.io/look-bench-page/](https://serendipityoneinc.github.io/look-bench-page/)

🤗 **Dataset**: [https://huggingface.co/datasets/srpone/look-bench](https://huggingface.co/datasets/srpone/look-bench)

## Setup and Installation

First, let's install the required packages and clone the LookBench repository.

In [ ]:
# Install required packages
!pip install -q torch torchvision transformers datasets pillow pandas pyarrow pyyaml tqdm matplotlib

# Clone LookBench repository
!git clone https://github.com/SerendipityOneInc/look-bench.git
%cd look-bench

print("✅ Installation complete!")

## 1. Load the LookBench Dataset from Hugging Face

LookBench dataset is hosted on Hugging Face and includes 4 subsets:
- **RealStudioFlat**: Easy, single-item retrieval from studio photos
- **AIGen-Studio**: Medium, AI-generated studio images
- **RealStreetLook**: Hard, multi-item outfit retrieval from street photos
- **AIGen-StreetLook**: Hard, AI-generated street outfits

In [ ]:
from datasets import load_dataset

# Load LookBench dataset from Hugging Face
print("Loading LookBench dataset...")
dataset = load_dataset("srpone/look-bench")

print(f"\nAvailable subsets: {list(dataset.keys())}")
print(f"\nDataset structure:")
for subset_name in dataset.keys():
    print(f"\n  {subset_name}:")
    print(f"    Splits: {list(dataset[subset_name].keys())}")
    for split_name in dataset[subset_name].keys():
        print(f"      {split_name}: {len(dataset[subset_name][split_name])} samples")

## 2. Explore a Sample

Let's look at a sample from the RealStudioFlat subset to understand the data structure.

In [ ]:
import matplotlib.pyplot as plt

# Get a sample from real_studio_flat query split
sample = dataset['real_studio_flat']['query'][0]

print("Sample keys:", list(sample.keys()))
print(f"\nCategory: {sample['category']}")
print(f"Main attribute: {sample['main_attribute']}")
print(f"Other attributes: {sample['other_attributes']}")
print(f"Task: {sample['task']}")
print(f"Difficulty: {sample['difficulty']}")

# Display the image
plt.figure(figsize=(6, 6))
plt.imshow(sample['image'])
plt.axis('off')
plt.title(f"{sample['category']} - {sample['main_attribute']}", fontsize=14, fontweight='bold')
plt.show()

## 3. Load a Model (CLIP)

Now let's load a pre-trained CLIP model for feature extraction.

In [ ]:
import torch
import sys
sys.path.append('/content/look-bench')

from manager import ConfigManager, ModelManager, DataManager

# Load configuration
config_manager = ConfigManager('configs/config.yaml')

# Create model manager
model_manager = ModelManager(config_manager)

print("Available models:", model_manager.get_available_models())

# Load CLIP model
print("\nLoading CLIP model...")
model, model_wrapper = model_manager.load_model('clip')
transform = model_manager.get_transform('clip')

# Set to evaluation mode
model.eval()

# Move to GPU if available
if torch.cuda.is_available():
    model = model.cuda()
    print("✅ Model moved to CUDA")
else:
    print("⚠️ Running on CPU (this will be slower)")

print("✅ Model loaded successfully!")

## 4. Extract Features from a Sample Image

Let's extract features from a sample image and see the feature representation.

In [ ]:
# Extract features from the sample image
sample_image = dataset['real_studio_flat']['query'][0]['image']
image_tensor = transform(sample_image).unsqueeze(0)

if torch.cuda.is_available():
    image_tensor = image_tensor.cuda()

with torch.no_grad():
    features = model(image_tensor)
    
print(f"Feature shape: {features.shape}")
print(f"Feature dimension: {features.shape[1]}")
print(f"Feature norm: {torch.norm(features).item():.4f}")
print(f"Feature min/max: [{features.min().item():.4f}, {features.max().item():.4f}]")

## 5. Compute Similarity with Gallery Images

Let's compute similarity between a query image and several gallery images to see how retrieval works.

In [ ]:
import numpy as np

def extract_features(images, model, transform):
    """Extract features from a list of images"""
    features = []
    for img in images:
        img_tensor = transform(img).unsqueeze(0)
        if torch.cuda.is_available():
            img_tensor = img_tensor.cuda()
        with torch.no_grad():
            feat = model(img_tensor)
        features.append(feat.cpu().numpy())
    return np.vstack(features)

# Get query image
query_img = dataset['real_studio_flat']['query'][0]['image']

# Get a few gallery images
gallery_imgs = [dataset['real_studio_flat']['gallery'][i]['image'] for i in range(5)]

# Extract features
print("Extracting query features...")
query_feat = extract_features([query_img], model, transform)

print("Extracting gallery features...")
gallery_feats = extract_features(gallery_imgs, model, transform)

# Compute cosine similarity
query_norm = query_feat / np.linalg.norm(query_feat, axis=1, keepdims=True)
gallery_norm = gallery_feats / np.linalg.norm(gallery_feats, axis=1, keepdims=True)
similarities = np.dot(query_norm, gallery_norm.T)[0]

print("\nSimilarity scores:")
for i, sim in enumerate(similarities):
    print(f"  Gallery image {i}: {sim:.4f}")

## 6. Visualize Query and Top Retrievals

Let's visualize the query image alongside the gallery images, ranked by similarity.

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 6, figsize=(18, 3))

# Show query
axes[0].imshow(query_img)
axes[0].set_title('Query', fontsize=12, fontweight='bold')
axes[0].axis('off')

# Show gallery images with similarity scores
for i, (img, sim) in enumerate(zip(gallery_imgs, similarities)):
    axes[i+1].imshow(img)
    axes[i+1].set_title(f'Gallery {i}\nSim: {sim:.3f}', fontsize=10)
    axes[i+1].axis('off')

plt.tight_layout()
plt.show()

print("\n✅ Quickstart completed successfully!")

## Next Steps

Congratulations! You've learned the basics of LookBench. Here's what you can do next:

1. **Full Evaluation**: Check out `02_model_evaluation.ipynb` to evaluate models on all metrics
2. **Custom Models**: See `03_custom_model.ipynb` to integrate your own models
3. **Explore Subsets**: Try different subsets (aigen_studio, real_streetlook, aigen_streetlook)
4. **Compare Models**: Evaluate different models (clip, siglip, dinov2)

### Useful Links

- 📄 **Paper**: https://arxiv.org/abs/2601.14706
- 🏠 **Project Page**: https://serendipityoneinc.github.io/look-bench-page/
- 🤗 **Dataset**: https://huggingface.co/datasets/srpone/look-bench
- 🤗 **GR-Lite Model**: https://huggingface.co/srpone/gr-lite
- 💻 **GitHub**: https://github.com/SerendipityOneInc/look-bench